In [359]:
import pandas as pd

In [360]:
csv_1 = pd.read_csv('GPT_II_1.csv', index_col=0, sep=',')[:3850]

In [361]:
csv_2 = pd.read_csv('GPT_II_2.csv', index_col=0, sep='|')
csv_2

,premise,hypothesis,rationales
3850,A small blond girl wearing a yellow shirt slid...,A female with clothes on is coming down a slid...,1. Premise: A small blond girl wearing a yello...
3851,A crowd of people look at something out of fra...,There is only 1 person,2. Premise: A crowd of people look at somethin...
3852,A man and a woman are standing together facing...,a man and a a woman are sleeping at home,3. Premise: A man and a woman are standing tog...
3853,A man in a black cap and white shirt is grilli...,A man cooking with gloves.,4. Premise: A man in a black cap and white shi...
3854,A girl is walking on a sidewalk with headphone...,A girl listens to Madonna while walking on a s...,5. Premise: A girl is walking on a sidewalk wi...
...,...,...,...
3995,A man in a white uniform is throwing another m...,A nurse practices throwing a patient onto a ta...,6. Premise: A man in a white uniform is throwi...
3996,People walking along the street with umbrellas.,people eat dogs,7. Premise: People walking along the street wi...
3997,Two bicyclists chat as they pass in front of a...,Two people with bikes are standing in front of...,8. Premise: Two bicyclists chat as they pass i...
3998,Someone fires a bullet from a handgun into the...,A guy shot a bullet from a gun into the trees.,9. Premise: Someone fires a bullet from a hand...


In [362]:
csv_3 = pd.read_csv('GPT_II_3.csv', index_col=0, sep='|')
csv_1 = pd.concat([csv_1, csv_2, csv_3])
csv_1

,premise,hypothesis,rationales
0,A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...
1,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...
2,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...
3,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...
4,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...
...,...,...,...
5587,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...
5588,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...
5589,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...
5590,Three naked little boys are playing in a river...,Three boys are playing in mud while one looks on.,Premise: Three naked little boys are playing i...


In [363]:
# from datasets import load_dataset
# import pandas as pd

# dataset = load_dataset("esnli")
# train_df = pd.DataFrame(dataset['train']).sample(frac=1/100, random_state=0)
# validation = pd.DataFrame(dataset['validation']).sample(frac=1/100, random_state=0)

# train_df = pd.concat([train_df, validation], ignore_index=True)

In [364]:
prompt_template= """Given the following premise and hypothesis, provide a detailed rationale that includes step-by-step reasoning leading to a conclusion of 'entailment', 'contradiction', or 'neutral'.
Premise: %s
Hypothesis: %s
Explain your reasoning."""

In [365]:
error = csv_1[csv_1.rationales.isna()]

In [366]:
error['prompt'] = error.apply(lambda x: prompt_template % (x['premise'], x['hypothesis']), axis=1)

In [367]:
error

,premise,hypothesis,rationales,prompt


In [368]:
print(error.prompt[:10].values)

[]


In [369]:
csv_1['split'] = csv_1.rationales.apply(lambda x: x.lower().split('most likely answer'))

In [370]:
csv_1.split = csv_1.split.apply(lambda x : x[1] if len(x) > 1 else x[0])

In [371]:
csv_1['correct_index'] = [{} for _ in range(len(csv_1))]
for index, row in csv_1.iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 'contradictory']:
        if choice in row['split']:
            row['correct_index'][choice] = row['split'].lower().index(choice)


In [378]:
csv_1[csv_1.LLM_answer.isna()]

for index, row in csv_1[csv_1.LLM_answer.isna()].iterrows():
    for choice in ["entailment", "neutral", "contradiction", 'entailed', 'contradicts', 'entails', 'contradicted', 
                   'contradictory', 'entail', 'contradict', 'supports', 'unrelated', 'discrepancy']:
        if choice in row['rationales'].lower():
            row.loc['correct_index'][choice] = row['rationales'].lower().index(choice)

In [379]:
import numpy as np
csv_1['LLM_answer'] =csv_1.correct_index.apply(lambda x: min(x, key=x.get) if len(x) > 0 else np.nan)

In [380]:
csv_1[csv_1.LLM_answer.isna()].index

Index([], dtype='int64')

In [744]:
# csv_1[csv_1.LLM_answer == 'entailed'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'entails'] = 'entailment'
# csv_1[csv_1.LLM_answer == 'contradicts'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradicted'] = 'contradiction'
# csv_1[csv_1.LLM_answer == 'contradictory'] = 'contradiction'

In [381]:
paper = pd.read_csv('../paper.csv', index_col=0)

In [382]:
csv_1['question'] = csv_1['premise'] + '</s>' + csv_1['hypothesis']
csv_1.set_index('question', inplace=True)

In [383]:
paper[~paper.premise.isin(csv_1.premise)]['hypothesis'].values

array([], dtype=object)

In [384]:
paper.shape

(5592, 6)

In [385]:
total_df = csv_1[(csv_1.premise.isin(paper.premise)) & (csv_1.hypothesis.isin(paper.hypothesis))].drop_duplicates(subset=['premise', 'hypothesis'])
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 589},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradicts': 446},contradicts
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'contradicts': 484},contradicts
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entails': 519},entails
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'entail': 470},entail
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,{'neutral': 585},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 563},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 564},entailment


In [386]:
total_df.loc[total_df.LLM_answer == 'entailed', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'entails', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradicts', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradicted', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'contradictory', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'entail', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'contradict', 'LLM_answer'] = 'contradiction'
total_df.loc[total_df.LLM_answer == 'supports', 'LLM_answer'] = 'entailment'
total_df.loc[total_df.LLM_answer == 'unrelated', 'LLM_answer'] = 'neutral'
total_df.loc[total_df.LLM_answer == 'discrepancy', 'LLM_answer'] = 'contradiction'

In [387]:
total_df

,premise,hypothesis,rationales,split,correct_index,LLM_answer
question,,,,,,
"A man with a white camp is standing on a platform in front of a large black cylinder device, and fabric on a clothesline.</s>A man is hanging off of a clothesline.",A man with a white camp is standing on a platf...,A man is hanging off of a clothesline.,1. Premise: A man with a white camp is standin...,1. premise: a man with a white camp is standin...,{'neutral': 589},neutral
Two girls wearing skirts like the flag of Great Britain.</s>a couple want to eat a flag.,Two girls wearing skirts like the flag of Grea...,a couple want to eat a flag.,2. Premise: Two girls wearing skirts like the ...,2. premise: two girls wearing skirts like the ...,{'contradicts': 446},contradiction
Dog leaps from water while woman kneels on the dock playing with him</s>A woman and dog sit on a dock together.,Dog leaps from water while woman kneels on the...,A woman and dog sit on a dock together.,3. Premise: Dog leaps from water while woman k...,3. premise: dog leaps from water while woman k...,{'contradicts': 484},contradiction
A bald man in an orange shirt and jeans is vacuuming an office carpet with a carry-along canister vacuum.</s>The clothed man is cleaning the carpet,A bald man in an orange shirt and jeans is vac...,The clothed man is cleaning the carpet,4. Premise: A bald man in an orange shirt and ...,4. premise: a bald man in an orange shirt and ...,{'entails': 519},entailment
A boy stands beside a railing at a go kart track.</s>The boy is going to ride a go kart.,A boy stands beside a railing at a go kart track.,The boy is going to ride a go kart.,5. Premise: A boy stands beside a railing at a...,5. premise: a boy stands beside a railing at a...,{'entail': 470},entailment
...,...,...,...,...,...,...
Three women walk down a city street; one looks inside her handbag as she walks.</s>Three women chat at home.,Three women walk down a city street; one looks...,Three women chat at home.,8. Premise: Three women walk down a city stree...,8. premise: three women walk down a city stree...,{'neutral': 585},neutral
A basketball player slam dunks the ball as his body goes sideways; as several people look on.</s>The ball player is competing in a slam dunk competition.,A basketball player slam dunks the ball as his...,The ball player is competing in a slam dunk co...,9. Premise: A basketball player slam dunks the...,9. premise: a basketball player slam dunks the...,{'entailment': 563},entailment
Two kids wearing white robes are practicing martial arts; boy on the left has blond-hair and is blocking a punch thrown by the dark-haired boy on the right.</s>The boys are practicing martial arts.,Two kids wearing white robes are practicing ma...,The boys are practicing martial arts.,10. Premise: Two kids wearing white robes are ...,10. premise: two kids wearing white robes are ...,{'entailment': 564},entailment


In [388]:
paper['my_label'] = total_df['LLM_answer']

In [389]:
sum(paper.label == paper.my_label) / len(paper)

0.6183834048640916

In [390]:
sum(paper.label == paper.llm_label) / len(paper)

0.699928469241774

In [935]:
total_df.to_csv('GPT_II - full.csv')